In [4]:
from dputils.scrape import Scraper, Tag

In [12]:
def get_url_list_from_amazon(url):
    sc = Scraper(url)
    urllist=sc.get_repeating_date_from_page(
        target = Tag('div', cls='s-result-list'),
        items = Tag('div',cls='s-result-item'),
        url=Tag('a', cls='a-link-normal',output='href'),
    )
    return [item['url']for item in urllist if item['url']]

In [11]:
url = 'https://www.amazon.in/s?k=mobile+5g&crid=284JTY1CYCYLQ&sprefix=mob%2Caps%2C317&ref=nb_sb_ss_ts-doa-p_8_3'
get_url_list_from_amazon(url)

[{'url': 'https://aax-eu.amazon.in/x/c/JN1-aJhxOV4h6olP3dxw3h8AAAGSKX1AoAMAAAH2AQBvbm9fdHhuX2JpZDMgICBvbm9fdHhuX2ltcDEgICBYlUGY/https://www.amazon.in/stores/page/D4FF818A-6C20-480D-960F-8ABAA178C71B/?_encoding=UTF8&store_ref=SB_A1034153KHL56M2RAWLC-A016707833EA2OADGJBS6&pd_rd_plhdr=t&aaxitk=067d11349f9fe49292c6a62faa753564&hsa_cr_id=0&lp_asins=B0DDTZF6G1%2CB0DDTXJ2MX%2CB0DDTXW9WH&lp_query=mobile%205g&lp_slot=auto-sparkle-hsa-tetris&ref_=sbx_be_s_sparkle_lsi4d_bkgd&pd_rd_w=64J1r&content-id=amzn1.sym.df9fe057-524b-4172-ac34-9a1b3c4e647d%3Aamzn1.sym.df9fe057-524b-4172-ac34-9a1b3c4e647d&pf_rd_p=df9fe057-524b-4172-ac34-9a1b3c4e647d&pf_rd_r=6W00B4Z8AVDNCPVCXVSJ&pd_rd_wg=uiiBf&pd_rd_r=6ae2698a-09b0-44e7-949b-cf228e35dffa'},
 {'url': None},
 {'url': '/sspa/click?ie=UTF8&spc=MTozMzg3ODU2Mjc1NjQxNzAxOjE3MjcyNzI5Mjc6c3BfYXRmOjMwMDE5MzUwNTg4ODQzMjo6MDo6&url=%2FiQOO-Storage-Snapdragon-6000mAh-FlashCharge%2Fdp%2FB07WFPMJBH%2Fref%3Dsr_1_1_sspa%3Fcrid%3D284JTY1CYCYLQ%26dib%3DeyJ2IjoiMSJ9.2vaHYCliE3wMV

database setup

In [17]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy import Column, Integer, String, Boolean

Base = declarative_base()

class Product(Base):
    __tablename__ = 'products'
    id = Column(Integer, primary_key=True)
    url = Column(String, unique=True)
    title = Column(String)
    sprice = Column(String)
    oprice = Column(String)
    reviews = Column(String)

engine = create_engine('sqlite:///amazon.db')
Base.metadata.create_all(engine)

def opendb():
    engine = create_engine('sqlite:///amazon.db')
    Session = sessionmaker(bind=engine)
    session = Session()
    return session

In [15]:
def get_details_from_url(product_url):
    if'html' in product_url:
        return None
    if product_url.startswith('/'):
        base_url ='https://www.amazon.in'
        product_url=base_url + product_url
    sc = Scraper(product_url)
    product = sc.get_data_from_page(
        name = Tag('h1', id='title'),
        sprice =Tag('span', cls='pricetopay'),
        oprice=Tag('span', cls='basisPrice'),
        revies = Tag('span', id='acrCustomerReviewText'),
    )
    return product

In [19]:
for product_url in get_url_list_from_amazon(url):
    product = get_details_from_url(product_url)
    if not product:
        continue
    if not product['name']:
        continue
    db = opendb()
    item = Product(
        url = product_url,
        title = product['name'].strip(),
        sprice = product['sprice'].strip(),
        oprice = product['oprice'].strip(),
        reviews = product['reviews'].strip(),
    )
    try:
        db.add(item)
        db.commit()
    except:
        print("Product details already in database")
    finally:
        db.close()
    print("Product details added to database")

AttributeError: 'NoneType' object has no attribute 'strip'